In [ ]:
#original file 

import numpy as np
 import pandas as pd
 import cupy as cp
 from numba import jit, cuda
 import time
 import matplotlib.pyplot as plt
 

 # --------------------------
 # 1. Data Loading and Preprocessing
 # --------------------------
 

 # Load historical high-frequency data (replace with your data source)
 # Data should at least contain: timestamp, bid_price, ask_price, trade_price, volume
 data = pd.read_csv('your_high_frequency_data.csv')
 data['timestamp'] = pd.to_datetime(data['timestamp'])
 data = data.sort_values('timestamp')
 

 # ---Resample data to a fixed interval (e.g., 100ms)---
 data = data.set_index('timestamp').resample('100L').agg({
  'bid_price': 'first',
  'ask_price': 'first',
  'trade_price': 'last',
  'volume': 'sum'
 })
 data = data.dropna()
 data = data.reset_index()
 

 # ---Normalize prices---
 initial_price = data['trade_price'][0]
 data['bid_price'] /= initial_price
 data['ask_price'] /= initial_price
 data['trade_price'] /= initial_price
 

 # ---Parameters---
 T = (data['timestamp'].max() - data['timestamp'].min()).total_seconds() # Total time
 dt = (data['timestamp'][1] - data['timestamp'][0]).total_seconds()
 N_t = len(data)
 S_0 = data['trade_price'][0]
 

 # ---Model Parameters---
 sigma = 0.2  # Volatility (can estimate from data)
 kappa = 0.001  # Inventory penalty
 gamma = 0.0001  # Transaction cost (as a fraction of trade)
 rho = 0.01  # Risk aversion
 market_impact_param = 0.0001
 inventory_max = 100
 

 # ---State Space Discretization---
 N_S = 101
 N_I = 2 * 51 - 1
 S_min = data['trade_price'].min() * 0.9
 S_max = data['trade_price'].max() * 1.1
 

 ds = (S_max - S_min) / (N_S - 1)
 di = inventory_max / (N_I // 2)
 

 S = np.linspace(S_min, S_max, N_S)
 I = np.linspace(-inventory_max, inventory_max, N_I)
 

 # ---Precompute for GPU---
 S_gpu = cp.asarray(S)
 I_gpu = cp.asarray(I)
 

 # --------------------------
 # 2. Order Book Simulation
 # --------------------------
 

 class OrderBook:
  def __init__(self):
  self.bids = {}  # {price: volume}
  self.asks = {}
 

  def process_order(self, price, volume, is_bid):
  if is_bid:
  self.bids[price] = self.bids.get(price, 0) + volume
  else:
  self.asks[price] = self.asks.get(price, 0) + volume
 

  def execute_market_order(self, volume, is_bid):
  executed_volume = 0
  cost = 0
  
  if is_bid:  # Buying
  ask_prices = sorted(self.asks.keys())
  for price in ask_prices:
  available_volume = self.asks[price]
  trade_volume = min(volume - executed_volume, available_volume)
  cost += trade_volume * price
  executed_volume += trade_volume
  self.asks[price] -= trade_volume
  if self.asks[price] == 0:
  del self.asks[price]
  if executed_volume == volume:
  break
  else:  # Selling
  bid_prices = sorted(self.bids.keys(), reverse=True)
  for price in bid_prices:
  available_volume = self.bids[price]
  trade_volume = min(volume - executed_volume, available_volume)
  cost += trade_volume * price
  executed_volume += trade_volume
  self.bids[price] -= trade_volume
  if self.bids[price] == 0:
  del self.bids[price]
  if executed_volume == volume:
  break
  
  if executed_volume < volume:
  print(f"Warning: Could not execute full order ({volume}), only executed {executed_volume}")
  
  return cost, executed_volume
 

  def get_best_bid_ask(self):
  best_bid = max(self.bids.keys()) if self.bids else 0
  best_ask = min(self.asks.keys()) if self.asks else float('inf')
  return best_bid, best_ask
 

 # --------------------------
 # 3. HJB Equation Solver
 # --------------------------
 

 # Utility function (risk aversion)
 def U(x, rho):
  return -cp.exp(-rho * x)
 

 @cuda.jit
 def HJB_step_gpu(V_next, V_current, S, I, dt, ds, di, sigma, kappa, gamma, rho,
  best_bid, best_ask, market_impact_param):
  s_idx, i_idx = cuda.grid(2)
 

  if s_idx > 0 and s_idx < S.shape[0] - 1 and i_idx > 0 and i_idx < I.shape[0] - 1:
  St = S[s_idx]
  It = I[i_idx]
 

  V_S_plus = V_next[s_idx + 1, i_idx]
  V_S_minus = V_next[s_idx - 1, i_idx]
  V_I_plus = V_next[s_idx, i_idx + 1]
  V_I_minus = V_next[s_idx, i_idx - 1]
 

  V_S = (V_S_plus - V_S_minus) / (2 * ds)
  V_SS = (V_S_plus - 2 * V_next[s_idx, i_idx] + V_S_minus) / (ds**2)
  V_I = (V_I_plus - V_I_minus) / (2 * di)
 

  # ---Market Maker's Control: Bid/Ask Spread---
  # ---This is a simplified control; in reality, it's a continuous optimization---
  
  # ---Possible actions: adjust bid/ask spread---
  spread_change = [-2 * ds, -ds, 0, ds, 2 * ds] # Possible spread changes
  
  V_optimal = -float('inf')
  
  for d_bid in spread_change:
  for d_ask in spread_change:
  
  new_bid = best_bid + d_bid
  new_ask = best_ask + d_ask
  
  if new_bid > 0 and new_ask > 0 and new_bid < new_ask: # Ensure valid prices
  
  # ---Simplified model of order arrival and execution---
  expected_buy = dt * 1 # Placeholder - Model order arrival better
  expected_sell = dt * 1
  
  expected_cost = (new_ask + market_impact_param * expected_buy) * expected_buy - \
  (new_bid - market_impact_param * expected_sell) * expected_sell
  
  V_trade = -gamma * expected_cost + V_next[s_idx, i_idx] - kappa * It**2 * dt + \
  dt * (0.5 * sigma**2 * St**2 * V_SS)
  
  V_optimal = max(V_optimal, V_trade)
  
  V_current[s_idx, i_idx] = V_optimal
 

 def solve_HJB_gpu(S_gpu, I_gpu, dt, ds, di, sigma, kappa, gamma, rho, U, N_t, data):
  V = cp.zeros((N_S, N_I, N_t))
  V[:, :, -1] = U(cp.zeros((N_S, N_I)), rho)  # Terminal condition
 

  threadsperblock = (8, 8)
  blockspergrid_x = int(np.ceil(V.shape[0] / threadsperblock[0]))
  blockspergrid_y = int(np.ceil(V.shape[1] / threadsperblock[1]))
  blockspergrid = (blockspergrid_x, blockspergrid_y)
 

  for n in range(N_t - 2, -1, -1):
  best_bid = data['bid_price'][n]
  best_ask = data['ask_price'][n]
  
  HJB_step_gpu[blockspergrid, threadsperblock](
  V[:, :, n + 1], V[:, :, n], S_gpu, I_gpu, dt, ds, di,
  sigma, kappa, gamma, rho, best_bid, best_ask, market_impact_param
 

  )
  return cp.asnumpy(V)
 

 # --------------------------
 # 4. Run the Solver
 # --------------------------
 

 start_time = time.time()
 V_optimal = solve_HJB_gpu(S_gpu, I_gpu, dt, ds, di, sigma, kappa, gamma, rho, U, N_t, data)
 end_time = time.time()
 print(f"HJB Solver Time: {end_time - start_time} seconds")
 

 # --------------------------
 # 5. Extract Policy and Simulate Trades
 # --------------------------
 

 def get_policy(V, S, I, data, ds):
  policy = np.zeros((V.shape[0], V.shape[1], V.shape[2], 2)) # [bid_change, ask_change]
  
  for t_idx in range(V.shape[2]):
  best_bid = data['bid_price'][t_idx]
  best_ask = data['ask_price'][t_idx]
  
  for s_idx in range(1, V.shape[0] - 1):
  for i_idx in range(1, V.shape[1] - 1):
  
  optimal_action = [0, 0] # [bid_change, ask_change]
  optimal_value = -float('inf')
  
  for d_bid in [-2 * ds, -ds, 0, ds, 2 * ds]:
  for d_ask in [-2 * ds, -ds, 0, ds, 2 * ds]:
  
  new_bid = best_bid + d_bid
  new_ask = best_ask + d_ask
  
  if new_bid > 0 and new_ask > 0 and new_bid < new_ask:
  
  value = V[s_idx, i_idx, t_idx] # Placeholder
  
  if value > optimal_value:
  optimal_value = value
  optimal_action = [d_bid, d_ask]
  
  policy[s_idx, i_idx, t_idx] = optimal_action
  
  return policy
 

 optimal_policy = get_policy(V_optimal, S, I, data, ds)
 

 def simulate_trades_with_order_book(data, S, I, policy, initial_wealth,
  initial_inventory, market_impact_param, gamma):
  wealth = initial_wealth
  inventory = initial_inventory
  order_book = OrderBook()
  
  wealth_history = [wealth]
  inventory_history = [inventory]
  bid_history = []
  ask_history = []
  
  for t_idx in range(len(data) - 1):
  
  # ---Get current market state---
  current_price = data['trade_price'][t_idx]
  
  # ---Find closest state indices---
  s_idx = np.argmin(np.abs(S - current_price))
  i_idx = np.argmin(np.abs(I - inventory))
  
  # ---Get action from policy---
  bid_change, ask_change = policy[min(s_idx, policy.shape[0] - 1),
  min(i_idx, policy.shape[1] - 1),
  t_idx]
  
  # ---Set market maker's quotes---
  best_bid, best_ask = order_book.get_best_bid_ask()
  if best_bid == 0:
  best_bid = data['bid_price'][t_idx]
  if best_ask == float('inf'):
  best_ask = data['ask_price'][t_idx]
  
  mm_bid = best_bid + bid_change
  mm_ask = best_ask + ask_change
  
  bid_history.append(mm_bid)
  ask_history.append(mm_ask)
  
  # ---Market orders from data---
  market_buy_volume = data['volume'][t_idx] if data['trade_price'][t_idx] > current_price else 0
  market_sell_volume = data['volume'][t_idx] if data['trade_price'][t_idx] < current_price else 0
  
  # ---Execute market maker trades---
  if mm_bid >= best_ask: # Market maker buys
  cost, executed_volume = order_book.execute_market_order(1, True)
  wealth -= cost * (1 + gamma)
  inventory += executed_volume
  elif mm_ask <= best_bid: # Market maker sells
  revenue, executed_volume = order_book.execute_market_order(1, False)
  wealth += revenue * (1 - gamma)
  inventory -= executed_volume
  else:
  order_book.process_order(mm_bid, 1, True)
  order_book.process_order(mm_ask, 1, False)
  
  # ---Update order book with market orders---
  order_book.execute_market_order(market_buy_volume, True)
  order_book.execute_market_order(market_sell_volume, False)
  
  wealth_history.append(wealth)
  inventory_history.append(inventory)
  
  return wealth_history, inventory_history, bid_history, ask_history
 

 # ---Backtesting Parameters---
 initial_wealth = 1000
 initial_inventory = 0
 

 wealth_history, inventory_history, bid_history, ask_history = simulate_trades_with_order_book(
  data, S, I, optimal_policy, initial_wealth, initial_inventory, market

# I am attempting to use the HJB Equation to learn more on the market_makers_optimisation_problem

In [19]:
import numpy as np
import pandas as pd
import cupy as cp
from numba import jit, cuda
import time
import matplotlib.pyplot as plt
from binance.client import Client
from datetime import datetime, timedelta
import os
import pytz
fetcher_client = Client(os.getenv('Binance_Fetcher_api'), os.getenv('Binance_Fetcher_secret')) #Main Data Fetcher API


AttributeError: 'ndarray' object has no attribute 'set_index'